In [17]:
# Importing Libraries

import torch as t
from torch.autograd import Variable
import numpy as np

In [18]:
# Calculation Of Saturation Pressure

p_sat14=10**(7.43155 - 1554.679/(20+240.337))
p_satw=10**(8.071 - 1730.63/(20+233.426))

p_g=[ 28.1 , 34.4 , 36.7 , 36.9 , 36.8 , 36.7 , 36.5 , 35.4 , 32.9 , 27.7 , 17.5 ]
x_g=[ 0.0 , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9 , 1.0 ]

In [19]:
# Saturation Pressure

print("Saturation Pressure of 1,4 dioxane is", p_sat14)
print("Saturation Pressure of water is", p_satw)

Saturation Pressure of 1,4 dioxane is 28.824099527405245
Saturation Pressure of water is 17.460784103526855


In [20]:
#Functions

# Pressure
def p(a,x1):
    x2=1-x1
    return x1 * t.exp(a[0]* ( (a[1]*x2)/(a[0]*x1 + a[1]*x2) )**2 ) * p_satw + x2 * t.exp(a[1]* ( (a[0]*x1)/(a[0]*x1 + a[1]*x2) )**2 ) * p_sat14

# Total sum
def TS(a):
    t=0
    for i in range(len(x_g)):
        xi=x_g[i]
        P=p(a,xi)
        t += (P-p_g[i])**2
    return t

# Step
def l(a):
    s=0.1
    while TS(a-s*a.grad) > TS(a)-s*(0)*np.matmul(a.grad, a.grad):
        s=.25*s
    return s

In [21]:
# Assuming initial values for A12 and A21 as 2 & 1.
a = Variable(t.tensor([2.0, 1.0]), requires_grad=True)

# Error
e = 150

# Gradient Descent
while e > 0.15:
    obj=TS(a)
    obj.backward()
    step=l(a)
    e = t.linalg.norm(a.grad)
    with t.no_grad():
        a -= step * a.grad
        a.grad.zero_()

print('Final value of a is ' + str(a.data.numpy()))
print('The value of objective function is ' + str(obj.data.numpy()))

Final value of a is [1.9594604 1.6896328]
The value of objective function is 0.67127305


In [22]:
# Comparison between given data and modelled data
print('Given P        Modelled P')
for i in range(0,11):
    print(str(p_g[i]) + '        ' + str(p(a.data,x_g[i]).item()))

Given P        Modelled P
28.1        28.824098587036133
34.4        34.64482498168945
36.7        36.45182800292969
36.9        36.86505126953125
36.8        36.87146759033203
36.7        36.747562408447266
36.5        36.3885498046875
35.4        35.38287353515625
32.9        32.94468688964844
27.7        27.723770141601562
17.5        17.460784912109375


The modelled pressures fit very well with the values given in the data.

In [1]:
def fun(x1,x2):
    return 1/((4 - 2.1*x1**2 + (x1**4)/3)*x1**2 + x1*x2 + (-4 + 4*(x2**2))*x2**2)

from bayes_opt import BayesianOptimization
bound = {'x1':(-3,3),'x2':(-2,2)}

optimizer = BayesianOptimization(f=fun,pbounds=bound,random_state=1)
optimizer.maximize(init_points=2,n_iter=100)
print(optimizer.max)

|   iter    |  target   |    x1     |    x2     |
-------------------------------------------------
|  1        | -3.773    | -0.4979   |  0.8813   |
|  2        |  0.009079 | -2.999    | -0.7907   |


TypeError: 'float' object is not subscriptable